In [1]:
Packages <- c("dplyr",  "nleqslv", "broom","cubature", "geosphere", "data.table",  "ggplot2", "bbmle", "stringr",  "lubridate", "RColorBrewer")

invisible(suppressPackageStartupMessages(lapply(Packages, library, character.only = TRUE)))

setwd('/local/home/katrinac/oceanography')
"%!in%" <- function(x,table) match(x,table, nomatch = 0) == 0
source("~/parentage/kernel_fitting/1340_loci/functions/ll_kt_both_bbmle.R")
source("~/parentage/kernel_fitting/1340_loci/functions/ll_kt_both_grid_search.R")
source("~/oceanography/scripts/neg_LL_biophys.R")
source("~/parentage/kernel_fitting/1340_loci/functions/GenGausKernInt_sum0.5.R") #integrate_kernel_sum1
source("~/parentage/kernel_fitting/1340_loci/functions/GenGausKernInt_sum1.R")
source("~/parentage/kernel_fitting/1340_loci/functions/cdf_solve.R") #median
source("~/parentage/kernel_fitting/1340_loci/functions/cdf_solve90.R") #dist 90% retained

#source("~/oceanography/scripts/PredictedProportions.R")

#read in the kernel fitting summary
kernels <- fread(file="~/parentage/kernel_fitting/1340_loci/final_results/tables/kernel_fitting_summary.csv")
kernel2012_14 <- fread(file="~/oceanography/empirical_data/genetics/GenKernelsForROMSComp2012-14.csv")

#read in the centroids adjusted for the simulation, so the Magbangons combined 
#centroids <- fread(file="~/oceanography/script_output/SurveyData/SimulationCentroids.csv")
Centroids <- fread(file="~/oceanography/empirical_data/site_centroids_SimTest.csv") 
Centroids$site <- gsub("_full", "", Centroids$site, fixed=TRUE)
Centroids$site <- gsub("_ten_per_cover", "", Centroids$site, fixed=TRUE)

Centroids <- Centroids %>%
    arrange(site)

#setorder(Centroids, site)#warning! This sets order based on site, and then lat/lon. So the table is not alphabetical by site, but that's fine as long as all of the "sampled_reef" vectors reflect this, so that reef_sizes, distance, and sampled_reefs match up by row/col index 
#read in the table with number of recruits sampled at each site for each year
AnnualRecsSamp <- fread(file="~/oceanography/script_output/SurveyData/AnnualRecruitsSampled.csv")
#read in the table of the proportion of anemones sampled at each site for each year
PropSamp <- unique(fread(file="~/oceanography/script_output/SurveyData/ProportionHabitatSampled.csv")[
    , .(site, year=end_year, prop_anem_samp=total_prop_hab_sampled_anems_tidied)][ #select and rename columns with the tideied data to use
    site %like% "Magbangon", site := "Magbangon"][ #collapse Magbangon values
    , prop_anem_samp := sum(prop_anem_samp), by=c("site", "year")], by=c("site", "year"))[ #collapse magbangons to match ROMS data
    site=="Sitio Lonas" & year %in% c(2012, 2013, 2014), prop_anem_samp :=1][site=="Caridad Proper" & year %in% c(2013, 2014), prop_anem_samp :=1]

### List of source locations
SitesSource <- Centroids

### List of destination locations
SitesDest <- Centroids

DistMatm <- distm(SitesSource[,c('lon','lat')], SitesSource[,c('lon','lat')], fun=distVincentyEllipsoid)
Distances <- DistMatm*10^-3
#read in the reef areas for the kernel fitting
Area <- fread("~/oceanography/empirical_data/site_area_header_nonsurveyed_simulation_kernels_test.csv") %>%
    arrange(site) %>%
    filter(site %!in% c("near_north_full1", "near_north_full2", "near_north_full3", "near_south_full1", "near_south_full2", "near_south_full3")) %>%
    mutate(kmsq=msq*10^-6)# %>%
    #select(kmsq) #need to uncomment for functions to work
Area$site <- gsub("_ten_per_cover", "", Area$site, fixed=TRUE)

reef_sizes <- as.matrix(Area$kmsq)

#make a site index table, use this for Sampled_reefs input in kernel fitting
SiteIndex <- unique(Centroids %>% arrange(site), by="site")[, index := .I] #add the row number as the unique site index, leave CAI in if fitting a kernel 
SiteIndexBioPhys <- unique(Centroids %>% arrange(site), by="site")[site != "CAI"][, index := .I] #add the row number as the unique site index, take CAI out for biophysical likelihood function

#make a table with the survey information for each site (how many fish sampled, prop anems sampled, total number of anems at site)
SurveyData <- AnnualRecsSamp[PropSamp, on=.(year, site)][#join the sampling tables together
    is.na(n_offs_gen), n_offs_gen := 0]#change NA's to 0
#setnames(SurveyData, c("PropAnemSamp", "TotalAnems"), c("prop_anem_samp", "total_anems"))
#setkey(SurveyData, site)
#check all sites are represented in centroids and area (and indirectly distances, which comes from centroids)
#Area[site %!in% centroids$site] #should be nothing

#Allison's abundance time series data 
#download.file(url = "https://github.com/pinskylab/Clownfish_persistence/blob/master/Data/Script_outputs/females_df_F.RData?raw=true", destfile = "~/oceanography/empirical_data/genetics/females_df_F.RData")
load("~/oceanography/empirical_data/genetics/females_df_F.RData")
Abundance <- as.data.table(females_df_F)
setnames(Abundance, "nF", "num_females")
Abundance <- unique(Abundance[site %like% "Magbangon", site := "Magbangon"][ #collapse Magbangon values
            , num_females := sum(num_females), by=c("site", "year")], by=c("site", "year"))
#join the survey sampling tables together
SurveyData <- AnnualRecsSamp[PropSamp, on=.(year, site)][
    is.na(n_offs_gen), n_offs_gen := 0]#change NA's to 0


SurveyData <- Abundance[, c("year", "site", "num_females")][SurveyData, on=.(year, site)]#join in Allison's estimate of female abundance. There are NA values, but that's okay we can figure those out when we start thinking about incorporating uncertainty in this
#quick check that all components are in the same, alphabetical order
sum(which(SiteIndex$site==Area$site)==FALSE) #needs to be 0!! sites have to be in the same order
sum(which(Area$site==Centroids$site)==FALSE) #needs to be 0!! sites have to be in the same order

#read in genetic parentage matrices for each time frame MATCHING DIMENSIONS WITH BIOPHYSICAL FOR BIOPHYSICAL LIKELIHOOD- NOT KERNEL FITTING
GenMat2012_4 <- as.matrix(fread(file="~/oceanography/script_output/SurveyData/20210625_ParentageMatrix2012-14ForROMSComp.csv"))
GenMat2012 <- as.matrix(fread(file="~/oceanography/script_output/SurveyData/20210625_ParentageMatrix2012ForROMSComp.csv"))
GenMat2013 <- as.matrix(fread(file="~/oceanography/script_output/SurveyData/20210625_ParentageMatrix2013ForROMSComp.csv"))
GenMat2014 <- as.matrix(fread(file="~/oceanography/script_output/SurveyData/20210625_ParentageMatrix2014ForROMSComp.csv"))
GenMatNEM <- as.matrix(fread(file="~/oceanography/script_output/SurveyData/20210625_ParentageMatrixNEM2012-14ForROMSComp.csv"))
GenMatSWM <- as.matrix(fread(file="~/oceanography/script_output/SurveyData/20210625_ParentageMatrixSWM2012-14ForROMSComp.csv"))

#read in genetic parentage matrices for each time frame FOR KERNEL FITTING
KernelGenMat2012_4 <- as.matrix(fread(file="~/oceanography/script_output/SurveyData/20210701_KernelParentageMatrix2012-14ForROMSComp.csv"))
KernelGenMat2012 <- as.matrix(fread(file="~/oceanography/script_output/SurveyData/20210701_KernelParentageMatrix2012ForROMSComp.csv"))
KernelGenMat2013 <- as.matrix(fread(file="~/oceanography/script_output/SurveyData/20210701_KernelParentageMatrix2013ForROMSComp.csv"))
KernelGenMat2014 <- as.matrix(fread(file="~/oceanography/script_output/SurveyData/20210701_KernelParentageMatrix2014ForROMSComp.csv"))
KernelGenMatNEM <- as.matrix(fread(file="~/oceanography/script_output/SurveyData/20210701_KernelParentageMatrixNEM2012-14ForROMSComp.csv"))
KernelGenMatSWM <- as.matrix(fread(file="~/oceanography/script_output/SurveyData/20210701_KernelParentageMatrixSWM2012-14ForROMSComp.csv"))

#read in biophysical data for biophysical likelihood
FullBiophysMatNorm <- as.matrix(fread(file="~/oceanography/script_output/ROMSDataTables/20210701_BioPhysNormConnMatrix2012-14ForROMSComp.csv"))
AnnualBiophysMatNorm2012 <- as.matrix(fread(file="~/oceanography/script_output/ROMSDataTables/20210701_BioPhysNormConnMatrix2012ForROMSComp.csv"))
AnnualBiophysMatNorm2013 <- as.matrix(fread(file="~/oceanography/script_output/ROMSDataTables/20210701_BioPhysNormConnMatrix2013ForROMSComp.csv"))
AnnualBiophysMatNorm2014 <- as.matrix(fread(file="~/oceanography/script_output/ROMSDataTables/20210701_BioPhysNormConnMatrix2014ForROMSComp.csv"))
MonsoonBiophysMatNormNEM <- as.matrix(fread(file="~/oceanography/script_output/ROMSDataTables/20210701_BioPhysNormConnMatrixNEM2012-14ForROMSComp.csv"))
MonsoonBiophysMatNormSWM <-as.matrix(fread(file="~/oceanography/script_output/ROMSDataTables/20210701_BioPhysNormConnMatrixSWM2012-14ForROMSComp.csv"))

#read in biophysical data for KERNEL FITTING
FullBiophysMat <- as.matrix(fread(file="~/oceanography/script_output/ROMSDataTables/20210701_BioPhysParentageMatrix2012-14ForROMSComp.csv"))
AnnualBiophysMat2012 <- as.matrix(fread(file="~/oceanography/script_output/ROMSDataTables/20210701_BioPhysParentageMatrix2012ForROMSComp.csv"))
AnnualBiophysMat2013 <- as.matrix(fread(file="~/oceanography/script_output/ROMSDataTables/20210701_BioPhysParentageMatrix2013ForROMSComp.csv"))
AnnualBiophysMat2014 <- as.matrix(fread(file="~/oceanography/script_output/ROMSDataTables/20210701_BioPhysParentageMatrix2014ForROMSComp.csv"))
MonsoonBiophysMatNEM <- as.matrix(fread(file="~/oceanography/script_output/ROMSDataTables/20210701_BioPhysParentageMatrixNEM2012-14ForROMSComp.csv"))
MonsoonBiophysMatSWM <- as.matrix(fread(file="~/oceanography/script_output/ROMSDataTables/20210701_BioPhysParentageMatrixSWM2012-14ForROMSComp.csv"))

[1] 0

[1] 0

In [ ]:
LL_kt

In [2]:


#fit the kernels, get the biophysical data together
biophys_par_data2012 <- list(Distances=Distances, Assignments=AnnualBiophysMat2012, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2012 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(AnnualBiophysMat2012), ncol=1, 1))
Sim2012Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=biophys_par_data2012, control=list(maxit=500)))

biophys_par_data2013 <- list(Distances=Distances, Assignments=AnnualBiophysMat2013, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2013 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(AnnualBiophysMat2013), ncol=1, 1))
Sim2013Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=biophys_par_data2013, control=list(maxit=500)))

biophys_par_data2014 <- list(Distances=Distances, Assignments=AnnualBiophysMat2014, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(AnnualBiophysMat2014), ncol=1, 1))
Sim2014Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=biophys_par_data2014, control=list(maxit=500)))

biophys_par_data2012_4 <- list(Distances=Distances, Assignments=FullBiophysMat, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(FullBiophysMat), ncol=1, 1))
Sim2012_4Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=biophys_par_data2012_4, control=list(maxit=500)))

biophys_par_dataNEM <- list(Distances=Distances, Assignments=MonsoonBiophysMatNEM, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(MonsoonBiophysMatNEM), ncol=1, 1))
SimNEMFit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=biophys_par_dataNEM, control=list(maxit=500)))

biophys_par_dataSWM <- list(Distances=Distances, Assignments=MonsoonBiophysMatSWM, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(MonsoonBiophysMatSWM), ncol=1, 1))
SimSWMFit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=biophys_par_dataSWM, control=list(maxit=500)))



In [8]:
summary(Sim2012Fit)
summary(Gen2012Fit)

Maximum likelihood estimation

Call:
mle2(minuslogl = LL_kt_bbmle, start = list(k = -3, theta = 1), 
    method = "L-BFGS-B", data = biophys_par_data2012, lower = c(-10, 
        0.15), upper = c(10, 8), control = list(maxit = 500))

Coefficients:
        Estimate Std. Error z value     Pr(z)    
k     -2.1643785  0.0027103 -798.58 < 2.2e-16 ***
theta  4.9907350  0.0474163  105.25 < 2.2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

-2 log L: 297166.3 

Maximum likelihood estimation

Call:
mle2(minuslogl = LL_kt_bbmle, start = list(k = -3, theta = 1), 
    method = "L-BFGS-B", data = par_data2012, lower = c(-10, 
        0.15), upper = c(10, 8), control = list(maxit = 500))

Coefficients:
      Estimate Std. Error z value  Pr(z)    
k     -3.27091    0.33591 -9.7375 <2e-16 ***
theta  3.52877    3.36407  1.0490 0.2942    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

-2 log L: 34.52999 

In [4]:

#make a summary table for all time frames of simulation data
SimKernels <- as.data.frame(matrix(NA, nrow = 6, ncol = 8))
setDT(SimKernels)
setnames(SimKernels, c("time_scale", "time_id", "k", "theta", "mdd", "med", "dist90", "log_like"))
#SimKernels <- data.table(time_scale=character(), time_id=character(), k=numeric(), theta=numeric(), mdd=numeric(), med=numeric(), dist90=numeric())


BestK2012 <- as.numeric(coef(Sim2012Fit)[1])
BestTheta2012 <- as.numeric(coef(Sim2012Fit)[2])
MDD2012 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2012, theta=BestTheta2012, , method = "pcubature")$integral)
k_eval <- BestK2012
theta_eval <- BestTheta2012
Med2012  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_2012 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)

SimKernels$time_scale[1] <- "annual"
SimKernels$time_id[1] <- "2012"
SimKernels$k[1] <-  BestK2012
SimKernels$theta[1] <- BestTheta2012
SimKernels$mdd[1] <- MDD2012
SimKernels$med[1] <- Med2012
SimKernels$dist90[1] <- Dist90_2012
SimKernels$log_like[1] <- logLik(Sim2012Fit)[1]

BestK2013 <- as.numeric(coef(Sim2013Fit)[1])
BestTheta2013 <- as.numeric(coef(Sim2013Fit)[2])
MDD2013 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2013, theta=BestTheta2013, , method = "pcubature")$integral)
k_eval <- BestK2013
theta_eval <- BestTheta2013
Med2013  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_2013 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)

SimKernels$time_scale[2] <- "annual"
SimKernels$time_id[2] <- "2013"
SimKernels$k[2] <-  BestK2013
SimKernels$theta[2] <- BestTheta2013
SimKernels$mdd[2] <- MDD2013
SimKernels$med[2] <- Med2013
SimKernels$dist90[2] <- Dist90_2013
SimKernels$log_like[2] <- logLik(Sim2013Fit)[1]


BestK2014 <- as.numeric(coef(Sim2014Fit)[1])
BestTheta2014 <- as.numeric(coef(Sim2014Fit)[2])
MDD2014 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2014, theta=BestTheta2014, , method = "pcubature")$integral)
k_eval <- BestK2014
theta_eval <- BestTheta2014
Med2014  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_2014 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)

SimKernels$time_scale[3] <- "annual"
SimKernels$time_id[3] <- "2014"
SimKernels$k[3] <-  BestK2014
SimKernels$theta[3] <- BestTheta2014
SimKernels$mdd[3] <- MDD2014
SimKernels$med[3] <- Med2014
SimKernels$dist90[3] <- Dist90_2014
SimKernels$log_like[3] <- logLik(Sim2014Fit)[1]

BestK2012_4 <- as.numeric(coef(Sim2012_4Fit)[1])
BestTheta2012_4 <- as.numeric(coef(Sim2012_4Fit)[2])
MDD2012_4 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2012_4, theta=BestTheta2012_4, , method = "pcubature")$integral)
k_eval <- BestK2012_4
theta_eval <- BestTheta2012_4
Med2012_4  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_2012_4 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)

SimKernels$time_scale[4] <- "interannual"
SimKernels$time_id[4] <- "2012-4"
SimKernels$k[4] <-  BestK2012_4
SimKernels$theta[4] <- BestTheta2012_4
SimKernels$mdd[4] <- MDD2012_4
SimKernels$med[4] <- Med2012_4
SimKernels$dist90[4] <- Dist90_2012_4
SimKernels$log_like[4] <- logLik(Sim2012_4Fit)[1]

BestKNEM <- as.numeric(coef(SimNEMFit)[1])
BestThetaNEM <- as.numeric(coef(SimNEMFit)[2])
MDDNEM <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestKNEM, theta=BestThetaNEM, , method = "pcubature")$integral)
k_eval <- BestKNEM
theta_eval <- BestThetaNEM
MedNEM  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_NEM <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)

SimKernels$time_scale[5] <- "season"
SimKernels$time_id[5] <- "NEM"
SimKernels$k[5] <-  BestKNEM
SimKernels$theta[5] <- BestThetaNEM
SimKernels$mdd[5] <- MDDNEM
SimKernels$med[5] <- MedNEM
SimKernels$dist90[5] <- Dist90_NEM
SimKernels$log_like[5] <- logLik(SimNEMFit)[1]

BestKSWM <- as.numeric(coef(SimSWMFit)[1])
BestThetaSWM <- as.numeric(coef(SimSWMFit)[2])
MDDSWM <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestKSWM, theta=BestThetaSWM, , method = "pcubature")$integral)
k_eval <- BestKSWM
theta_eval <- BestThetaSWM
MedSWM  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_SWM <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)

SimKernels$time_scale[6] <- "season"
SimKernels$time_id[6] <- "SWM"
SimKernels$k[6] <-  BestKSWM
SimKernels$theta[6] <- BestThetaSWM
SimKernels$mdd[6] <- MDDSWM
SimKernels$med[6] <- MedSWM
SimKernels$dist90[6] <- Dist90_SWM
SimKernels$log_like[6] <- logLik(SimSWMFit)[1]

SimKernels[, `:=`(k=round(k, 2), theta=round(theta, 2), mdd=round(mdd, 2), med=round(med, 2), dist90=round(dist90, 2), data_source="simulation"), by="time_id"]

#fwrite(SimKernels, file="~/oceanography/script_output/KernelFits/SimulationKernelFits.csv")


SimKernels


time_scale,time_id,k,theta,mdd,med,dist90,log_like,data_source
annual,2012,-2.16,4.99,4.21,4.01,7.88,-148583.15,simulation
annual,2013,-0.46,0.63,4.21,2.32,10.38,-549040.28,simulation
annual,2014,0.28,0.53,3.65,1.78,9.16,-527734.31,simulation
interannual,2012-4,-0.10,0.58,3.90,2.03,9.72,-1682004.75,simulation
season,NEM,-0.23,0.61,3.71,2.00,9.19,-1566546.34,simulation
season,SWM,0.60,0.42,7.89,3.17,20.02,-90949.14,simulation


In [3]:
#genetic data with same reef metadata structure as model

par_data2012 <- list(Distances=Distances, Assignments=KernelGenMat2012, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2012 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=as.matrix(SurveyData[year==2012 & prop_anem_samp >0, .(prop_anem_samp)]))
Gen2012Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=par_data2012, control=list(maxit=500)))

par_data2013 <- list(Distances=Distances, Assignments=KernelGenMat2013, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2013 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=as.matrix(SurveyData[year==2013 & prop_anem_samp >0, .(prop_anem_samp)]))
Gen2013Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=par_data2013, control=list(maxit=500)))

par_data2014 <- list(Distances=Distances, Assignments=KernelGenMat2014, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=as.matrix(SurveyData[year==2014 & prop_anem_samp >0, .(prop_anem_samp)]))
Gen2014Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=par_data2014, control=list(maxit=500)))

par_data2012_4 <- list(Distances=Distances, Assignments=KernelGenMat2012_4, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=as.matrix(SurveyData[year==2014 & prop_anem_samp >0, .(prop_anem_samp)]))
Gen2012_4Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=par_data2012_4, control=list(maxit=500)))

par_dataNEM <- list(Distances=Distances, Assignments=KernelGenMatNEM, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=as.matrix(SurveyData[year==2014 & prop_anem_samp >0, .(prop_anem_samp)]))
GenNEMFit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=par_dataNEM, control=list(maxit=500)))

par_dataSWM <- list(Distances=Distances, Assignments=KernelGenMatSWM, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=as.matrix(SurveyData[year==2014 & prop_anem_samp >0, .(prop_anem_samp)]))
GenSWMFit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=par_dataSWM, control=list(maxit=500)))



In [7]:
#make a summary table for all time frames of genetic data
GenKernels <- as.data.frame(matrix(NA, nrow = 6, ncol = 8))
setDT(GenKernels)
setnames(GenKernels, c("time_scale", "time_id", "k", "theta", "mdd", "med", "dist90", "log_like"))
#GenKernels <- data.table(time_scale=character(), time_id=character(), k=numeric(), theta=numeric(), mdd=numeric(), med=numeric(), dist90=numeric())


BestK2012 <- as.numeric(coef(Gen2012Fit)[1])
BestTheta2012 <- as.numeric(coef(Gen2012Fit)[2])
MDD2012 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2012, theta=BestTheta2012, , method = "pcubature")$integral)
k_eval <- BestK2012
theta_eval <- BestTheta2012
Med2012  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_2012 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)

GenKernels$time_scale[1] <- "annual"
GenKernels$time_id[1] <- "2012"
GenKernels$k[1] <-  BestK2012
GenKernels$theta[1] <- BestTheta2012
GenKernels$mdd[1] <- MDD2012
GenKernels$med[1] <- Med2012
GenKernels$dist90[1] <- Dist90_2012
GenKernels$log_like[1] <- logLik(Gen2012Fit)[1]

BestK2013 <- as.numeric(coef(Gen2013Fit)[1])
BestTheta2013 <- as.numeric(coef(Gen2013Fit)[2])
MDD2013 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2013, theta=BestTheta2013, , method = "pcubature")$integral)
k_eval <- BestK2013
theta_eval <- BestTheta2013
Med2013  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_2013 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)

GenKernels$time_scale[2] <- "annual"
GenKernels$time_id[2] <- "2013"
GenKernels$k[2] <-  BestK2013
GenKernels$theta[2] <- BestTheta2013
GenKernels$mdd[2] <- MDD2013
GenKernels$med[2] <- Med2013
GenKernels$dist90[2] <- Dist90_2013
GenKernels$log_like[2] <- logLik(Gen2013Fit)[1]

BestK2014 <- as.numeric(coef(Gen2014Fit)[1])
BestTheta2014 <- as.numeric(coef(Gen2014Fit)[2])
MDD2014 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2014, theta=BestTheta2014, , method = "pcubature")$integral)
k_eval <- BestK2014
theta_eval <- BestTheta2014
Med2014  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_2014 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)

GenKernels$time_scale[3] <- "annual"
GenKernels$time_id[3] <- "2014"
GenKernels$k[3] <-  BestK2014
GenKernels$theta[3] <- BestTheta2014
GenKernels$mdd[3] <- MDD2014
GenKernels$med[3] <- Med2014
GenKernels$dist90[3] <- Dist90_2014
GenKernels$log_like[3] <- logLik(Gen2014Fit)[1]

BestK2012_4 <- as.numeric(coef(Gen2012_4Fit)[1])
BestTheta2012_4 <- as.numeric(coef(Gen2012_4Fit)[2])
MDD2012_4 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2012_4, theta=BestTheta2012_4, , method = "pcubature")$integral)
k_eval <- BestK2012_4
theta_eval <- BestTheta2012_4
Med2012_4  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_2012_4 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)

GenKernels$time_scale[4] <- "interannual"
GenKernels$time_id[4] <- "2012-4"
GenKernels$k[4] <-  BestK2012_4
GenKernels$theta[4] <- BestTheta2012_4
GenKernels$mdd[4] <- MDD2012_4
GenKernels$med[4] <- Med2012_4
GenKernels$dist90[4] <- Dist90_2012_4
GenKernels$log_like[4] <- logLik(Gen2012_4Fit)[1]

BestKNEM <- as.numeric(coef(GenNEMFit)[1])
BestThetaNEM <- as.numeric(coef(GenNEMFit)[2])
MDDNEM <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestKNEM, theta=BestThetaNEM, , method = "pcubature")$integral)
k_eval <- BestKNEM
theta_eval <- BestThetaNEM
MedNEM  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_NEM <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)

GenKernels$time_scale[5] <- "season"
GenKernels$time_id[5] <- "NEM"
GenKernels$k[5] <-  BestKNEM
GenKernels$theta[5] <- BestThetaNEM
GenKernels$mdd[5] <- MDDNEM
GenKernels$med[5] <- MedNEM
GenKernels$dist90[5] <- Dist90_NEM
GenKernels$log_like[5] <- logLik(GenNEMFit)[1]

BestKSWM <- as.numeric(coef(GenSWMFit)[1])
BestThetaSWM <- as.numeric(coef(GenSWMFit)[2])
MDDSWM <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestKSWM, theta=BestThetaSWM, , method = "pcubature")$integral)
k_eval <- BestKSWM
theta_eval <- BestThetaSWM
MedSWM  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_SWM <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)

GenKernels$time_scale[6] <- "season"
GenKernels$time_id[6] <- "SWM"
GenKernels$k[6] <-  BestKSWM
GenKernels$theta[6] <- BestThetaSWM
GenKernels$mdd[6] <- MDDSWM
GenKernels$med[6] <- MedSWM
GenKernels$dist90[6] <- Dist90_SWM
GenKernels$log_like[6] <- logLik(GenSWMFit)[1]

GenKernels[, `:=`(k=round(k, 2), theta=round(theta, 2), mdd=round(mdd, 2), med=round(med, 2), dist90=round(dist90, 2), data_source="genetics"), by="time_id"]

#fwrite(GenKernels, file="~/oceanography/script_output/KernelFits/GeneticKernelFits.csv")


GenKernels



time_scale,time_id,k,theta,mdd,med,dist90,log_like,data_source
annual,2012,-3.27,3.53,13.02,12.02,25.08,-17.26499,genetics
annual,2013,-3.38,8.00,14.20,13.89,25.94,-111.36299,genetics
annual,2014,1.97,0.26,76.17,17.50,182.75,-78.48620,genetics
interannual,2012-4,-0.23,0.44,13.69,5.77,34.74,-218.61429,genetics
season,NEM,0.76,0.37,12.87,4.64,32.61,-27.26112,genetics
season,SWM,-3.46,8.00,15.35,15.02,28.05,-70.92640,genetics


In [ ]:
# use a grid search to find k that minimizes the log likelihood
LogLikeSim2012 <- setDT(expand.grid(k_eval=seq(from=-10, to=10, length.out = 2000), theta_eval=seq(from=0.1, to=5, length.out = 200)))
LogLikeSim2012[, log_like := LL_kt_grid(k=k_eval, theta=theta_eval, Distances=biophys_par_data2012$Distances, Assignments=biophys_par_data2012$Assignments, Sampled_reefs=biophys_par_data2012$Sampled_reefs, Reef_sizes=biophys_par_data2012$Reef_sizes, Adult_sample_proportions=biophys_par_data2012$Adult_sample_proportions), by=.(k_eval, theta_eval)]   #write profile results
fwrite(LogLikeSim2012, file="~/oceanography/script_output/KernelFits/LikelihoodProfileBiophysical2012.csv")

LogLikeSim2013 <- setDT(expand.grid(k_eval=seq(from=-10, to=10, length.out = 2000), theta_eval=seq(from=0.1, to=5, length.out = 200)))
LogLikeSim2013[, log_like := LL_kt_grid(k=k_eval, theta=theta_eval, Distances=biophys_par_data2013$Distances, Assignments=biophys_par_data2013$Assignments, Sampled_reefs=biophys_par_data2013$Sampled_reefs, Reef_sizes=biophys_par_data2013$Reef_sizes, Adult_sample_proportions=biophys_par_data2013$Adult_sample_proportions), by=.(k_eval, theta_eval)] 
#write profile results
fwrite(LogLikeSim2013, file="~/oceanography/script_output/KernelFits/LikelihoodProfileBiophysical2013.csv")

LogLikeSim2014 <- setDT(expand.grid(k_eval=seq(from=-10, to=10, length.out = 2000), theta_eval=seq(from=0.1, to=5, length.out = 200)))
LogLikeSim2014[, log_like := LL_kt_grid(k=k_eval, theta=theta_eval, Distances=biophys_par_data2014$Distances, Assignments=biophys_par_data2014$Assignments, Sampled_reefs=biophys_par_data2014$Sampled_reefs, Reef_sizes=biophys_par_data2014$Reef_sizes, Adult_sample_proportions=biophys_par_data2014$Adult_sample_proportions), by=.(k_eval, theta_eval)] 
#write profile results
fwrite(LogLikeSim2014, file="~/oceanography/script_output/KernelFits/LikelihoodProfileBiophysical2014.csv")

LogLikeSim2012_4 <- setDT(expand.grid(k_eval=seq(from=-10, to=10, length.out = 2000), theta_eval=seq(from=0.1, to=5, length.out = 200)))
LogLikeSim2012_4[, log_like := LL_kt_grid(k=k_eval, theta=theta_eval, Distances=biophys_par_data2012_4$Distances, Assignments=biophys_par_data2012_4$Assignments, Sampled_reefs=biophys_par_data2012_4$Sampled_reefs, Reef_sizes=biophys_par_data2012_4$Reef_sizes, Adult_sample_proportions=biophys_par_data2012_4$Adult_sample_proportions), by=.(k_eval, theta_eval)] 
#write profile results
fwrite(LogLikeSim2012_4, file="~/oceanography/script_output/KernelFits/LikelihoodProfileBiophysical2012_4.csv")

LogLikeSimNEM <- setDT(expand.grid(k_eval=seq(from=-10, to=10, length.out = 2000), theta_eval=seq(from=0.1, to=5, length.out = 200)))
LogLikeSimNEM[, log_like := LL_kt_grid(k=k_eval, theta=theta_eval, Distances=biophys_par_dataNEM$Distances, Assignments=biophys_par_dataNEM$Assignments, Sampled_reefs=biophys_par_dataNEM$Sampled_reefs, Reef_sizes=biophys_par_dataNEM$Reef_sizes, Adult_sample_proportions=biophys_par_dataNEM$Adult_sample_proportions), by=.(k_eval, theta_eval)] 
#write profile results
fwrite(LogLikeSimNEM, file="~/oceanography/script_output/KernelFits/LikelihoodProfileBiophysicalNEM.csv")

LogLikeSimSWM <- setDT(expand.grid(k_eval=seq(from=-10, to=10, length.out = 2000), theta_eval=seq(from=0.1, to=5, length.out = 200)))
LogLikeSimSWM[, log_like := LL_kt_grid(k=k_eval, theta=theta_eval, Distances=biophys_par_dataSWM$Distances, Assignments=biophys_par_dataSWM$Assignments, Sampled_reefs=biophys_par_dataSWM$Sampled_reefs, Reef_sizes=biophys_par_dataSWM$Reef_sizes, Adult_sample_proportions=biophys_par_dataSWM$Adult_sample_proportions), by=.(k_eval, theta_eval)] 
#write profile results
fwrite(LogLikeSimSWM, file="~/oceanography/script_output/KernelFits/LikelihoodProfileBiophysicalSWM.csv")

#also for genetics
LogLikeGen2012 <- setDT(expand.grid(k_eval=seq(from=-10, to=10, length.out = 2000), theta_eval=seq(from=0.1, to=5, length.out = 200)))
LogLikeGen2012[, log_like := LL_kt_grid(k=k_eval, theta=theta_eval, Distances=biophys_par_data2012$Distances, Assignments=biophys_par_data2012$Assignments, Sampled_reefs=biophys_par_data2012$Sampled_reefs, Reef_sizes=biophys_par_data2012$Reef_sizes, Adult_sample_proportions=biophys_par_data2012$Adult_sample_proportions), by=.(k_eval, theta_eval)] 
#write profile results
fwrite(LogLikeGen2012, file="~/oceanography/script_output/KernelFits/LikelihoodProfileGenetic2012.csv")

LogLikeGen2013 <- setDT(expand.grid(k_eval=seq(from=-10, to=10, length.out = 2000), theta_eval=seq(from=0.1, to=5, length.out = 200)))
LogLikeGen2013[, log_like := LL_kt_grid(k=k_eval, theta=theta_eval, Distances=biophys_par_data2013$Distances, Assignments=biophys_par_data2013$Assignments, Sampled_reefs=biophys_par_data2013$Sampled_reefs, Reef_sizes=biophys_par_data2013$Reef_sizes, Adult_sample_proportions=biophys_par_data2013$Adult_sample_proportions), by=.(k_eval, theta_eval)] 
#write profile results
fwrite(LogLikeGen2013, file="~/oceanography/script_output/KernelFits/LikelihoodProfileGenetic2013.csv")

LogLikeGen2014 <- setDT(expand.grid(k_eval=seq(from=-10, to=10, length.out = 2000), theta_eval=seq(from=0.1, to=5, length.out = 200)))
LogLikeGen2014[, log_like := LL_kt_grid(k=k_eval, theta=theta_eval, Distances=biophys_par_data2014$Distances, Assignments=biophys_par_data2014$Assignments, Sampled_reefs=biophys_par_data2014$Sampled_reefs, Reef_sizes=biophys_par_data2014$Reef_sizes, Adult_sample_proportions=biophys_par_data2014$Adult_sample_proportions), by=.(k_eval, theta_eval)] 
#write profile results
fwrite(LogLikeGen2014, file="~/oceanography/script_output/KernelFits/LikelihoodProfileGenetic2014.csv")

LogLikeGen2012_4 <- setDT(expand.grid(k_eval=seq(from=-10, to=10, length.out = 2000), theta_eval=seq(from=0.1, to=5, length.out = 200)))
LogLikeGen2012_4[, log_like := LL_kt_grid(k=k_eval, theta=theta_eval, Distances=biophys_par_data2012_4$Distances, Assignments=biophys_par_data2012_4$Assignments, Sampled_reefs=biophys_par_data2012_4$Sampled_reefs, Reef_sizes=biophys_par_data2012_4$Reef_sizes, Adult_sample_proportions=biophys_par_data2012_4$Adult_sample_proportions), by=.(k_eval, theta_eval)] 
#write profile results
fwrite(LogLikeGen2012_4, file="~/oceanography/script_output/KernelFits/LikelihoodProfileGenetic2012_4.csv")

LogLikeGenNEM <- setDT(expand.grid(k_eval=seq(from=-10, to=10, length.out = 2000), theta_eval=seq(from=0.1, to=5, length.out = 200)))
LogLikeGenNEM[, log_like := LL_kt_grid(k=k_eval, theta=theta_eval, Distances=biophys_par_dataNEM$Distances, Assignments=biophys_par_dataNEM$Assignments, Sampled_reefs=biophys_par_dataNEM$Sampled_reefs, Reef_sizes=biophys_par_dataNEM$Reef_sizes, Adult_sample_proportions=biophys_par_dataNEM$Adult_sample_proportions), by=.(k_eval, theta_eval)] 
#write profile results
fwrite(LogLikeGenNEM, file="~/oceanography/script_output/KernelFits/LikelihoodProfileGeneticNEM.csv")

LogLikeGenSWM <- setDT(expand.grid(k_eval=seq(from=-10, to=10, length.out = 2000), theta_eval=seq(from=0.1, to=5, length.out = 200)))
LogLikeGenSWM[, log_like := LL_kt_grid(k=k_eval, theta=theta_eval, Distances=biophys_par_dataSWM$Distances, Assignments=biophys_par_dataSWM$Assignments, Sampled_reefs=biophys_par_dataSWM$Sampled_reefs, Reef_sizes=biophys_par_dataSWM$Reef_sizes, Adult_sample_proportions=biophys_par_dataSWM$Adult_sample_proportions), by=.(k_eval, theta_eval)] 
#write profile results
fwrite(LogLikeGenSWM, file="~/oceanography/script_output/KernelFits/LikelihoodProfileGeneticSWM.csv")

In [ ]:
##lay out all the pieces
#sampled_reefs_vec <- as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)])
#pop_size_vec <- as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]) #vector of pop sizes for all reefs (a). This term is also used in parentage kernel fitting, but reef sizes are substituted as a proxy for pop size. This is should be bootstrapped to account for uncertainty.
#BioPhysMat <- as.matrix(FullBiophysMatNorm[1:nrow(sampled_reefs_vec),]) #source normalized biophysical connectivity matrix. In Eqn. S3.4, this is m ajt/r a (*should it be r at? As in all particles released in time period t?)
#prop_samp_vec <- as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)])#vector of proportion of habitat sampled for all reefs in time period t- I think that I should be coming up with a different value for this, but for now this will help me code the function
#unassigned_vec <- as.matrix(GenMat2012_4[nrow(GenMat2012_4),])#from genetic parentage data- a vector of the number of unassigned recruits at each destination reef in the system- we only have this for all sampled reefs.... what should the dimensions be?*
#Assignments <- GenMat2012_4[1:nrow(GenMat2012_4)-1,]
#
Data2012_4 <- list(BioPhysMat=as.matrix(FullBiophysMatNorm[1:nrow(sampled_reefs_vec),]),
                    Assignments=GenMat2012_4[1:nrow(GenMat2012_4)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(GenMat2012_4[nrow(GenMat2012_4),]))

Data2012 <- list(BioPhysMat=as.matrix(AnnualBiophysMatNorm2012[1:nrow(sampled_reefs_vec),]),
                    Assignments=GenMat2012[1:nrow(GenMat2012)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(GenMat2012[nrow(GenMat2012),]))
Data2013 <- list(BioPhysMat=as.matrix(AnnualBiophysMatNorm2013[1:nrow(sampled_reefs_vec),]),
                    Assignments=GenMat2013[1:nrow(GenMat2013)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(GenMat2013[nrow(GenMat2013),]))
 Data2014 <- list(BioPhysMat=as.matrix(AnnualBiophysMatNorm2014[1:nrow(sampled_reefs_vec),]),
                    Assignments=GenMat2014[1:nrow(GenMat2014)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(GenMat2014[nrow(GenMat2014),]))

 DataNEM <- list(BioPhysMat=as.matrix(MonsoonBiophysMatNormNEM[1:nrow(sampled_reefs_vec),]),
                    Assignments=GenMat2012_4[1:nrow(GenMat2012_4)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(GenMatNEM[nrow(GenMatNEM),]))

 DataSWM <- list(BioPhysMat=as.matrix(MonsoonBiophysMatNormSWM[1:nrow(sampled_reefs_vec),]),
                    Assignments=GenMat2012_4[1:nrow(GenMat2012_4)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(GenMatSWM[nrow(GenMatSWM),]))

#calculate likelihood of our observed data given the biophysical simulation data- we want the highest likelihood, which is the smallest negative log likelihood
data.table(time_frame= c("interannual", "annual", "monsoonal"), neg_LL=c(neg_LL_biophys(Data2012_4), neg_LL_biophys(Data2012) + neg_LL_biophys(Data2013) + neg_LL_biophys(Data2014), neg_LL_biophys(DataNEM) + neg_LL_biophys(DataSWM)))[order(-neg_LL)]


Data2012_test <- list(BioPhysMat=as.matrix(FullBiophysMatNorm[1:nrow(sampled_reefs_vec),]),
                    Assignments=GenMat2012[1:nrow(GenMat2012)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2012,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(GenMat2012[nrow(GenMat2012),]))
Data2013_test <- list(BioPhysMat=as.matrix(FullBiophysMatNorm[1:nrow(sampled_reefs_vec),]),
                    Assignments=GenMat2013[1:nrow(GenMat2013)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2013,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(GenMat2013[nrow(GenMat2013),]))
Data2014_test <- list(BioPhysMat=as.matrix(FullBiophysMatNorm[1:nrow(sampled_reefs_vec),]),
                    Assignments=GenMat2014[1:nrow(GenMat2014)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(GenMat2014[nrow(GenMat2014),]))

neg_LL_biophys(Data2012_test)+neg_LL_biophys(Data2013_test)+neg_LL_biophys(Data2014_test)

 DataNEM_test <- list(BioPhysMat=as.matrix(FullBiophysMatNorm[1:nrow(sampled_reefs_vec),]),
                    Assignments=GenMatNEM[1:nrow(GenMatNEM)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(GenMatNEM[nrow(GenMatNEM),]))

 DataSWM_test <- list(BioPhysMat=as.matrix(FullBiophysMatNorm[1:nrow(sampled_reefs_vec),]),
                    Assignments=GenMatSWM[1:nrow(GenMatSWM)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(GenMatSWM[nrow(GenMatSWM),]))
neg_LL_biophys(DataNEM_test)+neg_LL_biophys( DataSWM_test)

sum(colSums(MonsoonBiophysMatNormNEM[1:nrow(sampled_reefs_vec),]))
sum(colSums(MonsoonBiophysMatNormSWM[1:nrow(sampled_reefs_vec),]))

